In [1]:
#%matplotlib qt
from keras.models import Sequential, load_model
from keras.datasets import imdb
from keras.layers import Dense, Dropout
from keras.layers import LSTM, RNN, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
#import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
pos_set = np.load("./train/pos_set.npy")
neg_set = np.load("./train/neg_set.npy")

complete = np.concatenate((pos_set, neg_set))
y = np.concatenate((np.ones((pos_set.shape[0],1)), np.zeros((neg_set.shape[0],1))))

scaled_data = []

for trajectory in complete:
    std = StandardScaler()
    scaled_data.append(std.fit_transform(trajectory))

scaled_data = np.asarray(scaled_data)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float128 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
scaled_data.shape, y.shape

((1933, 999, 2), (1933, 1))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.25, random_state=42)

In [17]:
model = Sequential()
model.add(LSTM(126, input_shape=(999, 2), return_sequences=True, dropout=0.2))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(16, dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 999, 126)          65016     
_________________________________________________________________
lstm_8 (LSTM)                (None, 999, 64)           48896     
_________________________________________________________________
lstm_9 (LSTM)                (None, 16)                5184      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 119,113
Trainable params: 119,113
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

In [41]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_split=0.3,
                    shuffle=True,
                    callbacks=[ModelCheckpoint("./models_lstm1/weights.{epoch:02d}-{val_loss:.2f}.hdf5")]
                   )

Train on 1014 samples, validate on 435 samples
Epoch 1/10
1014/1014 [==============================] - 68s 67ms/step - loss: 0.1371 - acc: 0.9645 - val_loss: 0.0437 - val_acc: 0.9977
Epoch 2/10
1014/1014 [==============================] - 67s 66ms/step - loss: 0.1347 - acc: 0.9645 - val_loss: 0.0438 - val_acc: 0.9954
Epoch 3/10
1014/1014 [==============================] - 67s 67ms/step - loss: 0.1928 - acc: 0.9310 - val_loss: 0.1147 - val_acc: 0.9379
Epoch 4/10
1014/1014 [==============================] - 68s 67ms/step - loss: 0.2212 - acc: 0.9142 - val_loss: 0.0943 - val_acc: 0.9770
Epoch 5/10
1014/1014 [==============================] - 68s 67ms/step - loss: 0.1615 - acc: 0.9467 - val_loss: 0.1256 - val_acc: 0.9701
Epoch 6/10
1014/1014 [==============================] - 68s 67ms/step - loss: 0.1098 - acc: 0.9655 - val_loss: 0.0526 - val_acc: 0.9977
Epoch 7/10
1014/1014 [==============================] - 67s 67ms/step - loss: 0.1188 - acc: 0.9625 - val_loss: 0.2481 - val_acc: 0.9540
E

In [42]:
model.evaluate(X_test,y_test)

484/484 [==============================] - 5s 11ms/step


[0.040990727638040694, 0.993801652892562]

In [85]:
plt.plot(range(1,11),history.history["val_acc"], label="Validation Accuracy")
plt.plot(range(1,11), history.history["acc"], label="Training Accuracy")

NameError: name 'history' is not defined

In [9]:
model_train_test = load_model("./models_lstm1/weights.10-0.05.hdf5")

In [74]:
seed = 42
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(scaled_data, y):
  # create model
    print("making model")
    model = Sequential()
    model.add(LSTM(128, input_shape=(999, 2), return_sequences=True, dropout=0.2))
    model.add(LSTM(32, return_sequences=True, dropout=0.2))
    model.add(LSTM(16))
    model.add(Dense(1, activation="sigmoid"))
    # Compile model
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # Fit the model
    print("training model")
    history = model.fit(scaled_data[train], y[train], epochs=4)
    # evaluate the model
    print("evaluating model")
    scores = model.evaluate(scaled_data[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("training accuracy: %.2f" %(history.history["acc"])[1])
    cvscores.append(scores[1] * 100)
    print("\n\n")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

making model
training model
Epoch 1/4
1739/1739 [==============================] - 183s 105ms/step - loss: 0.5643 - acc: 0.7269
Epoch 2/4
1739/1739 [==============================] - 180s 103ms/step - loss: 0.4589 - acc: 0.8079
Epoch 3/4
1739/1739 [==============================] - 179s 103ms/step - loss: 0.5636 - acc: 0.7430
Epoch 4/4
1739/1739 [==============================] - 177s 102ms/step - loss: 0.3950 - acc: 0.8419
evaluating model
acc: 97.94%
training accuracy: 0.81



making model
training model
Epoch 1/4
1739/1739 [==============================] - 182s 105ms/step - loss: 0.5192 - acc: 0.7660
Epoch 2/4
1739/1739 [==============================] - 177s 102ms/step - loss: 0.3727 - acc: 0.8534
Epoch 3/4
1739/1739 [==============================] - 178s 102ms/step - loss: 0.3029 - acc: 0.8930
Epoch 4/4
1739/1739 [==============================] - 179s 103ms/step - loss: 0.3013 - acc: 0.8976
evaluating model
acc: 98.97%
training accuracy: 0.85



making model
training model
Epoc

In [5]:
model_cnn = Sequential()
model_cnn.add(Conv1D(64, 7, activation="relu", input_shape=(999,2)))
model_cnn.add(MaxPooling1D(5))
model_cnn.add(Conv1D(64, 7, activation="relu"))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(1, activation="sigmoid"))

In [6]:
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 993, 64)           960       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 198, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 192, 64)           28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 29,761
Trainable params: 29,761
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_cnn.compile(optimizer="rmsprop",
    loss='binary_crossentropy',
    metrics=['acc'])


In [11]:
history_cnn = model_cnn.fit(X_train, y_train,
    epochs=10,
    validation_split=0.3,
    shuffle=True,
    callbacks=[ModelCheckpoint("./models_1Dcnn/weights.{epoch:02d}-{val_loss:.2f}.hdf5")])

Train on 1014 samples, validate on 435 samples
Epoch 1/10
1014/1014 [==============================] - 5s 5ms/step - loss: 0.0073 - acc: 1.0000 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 2/10
1014/1014 [==============================] - 5s 5ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 7.5493e-04 - val_acc: 1.0000
Epoch 3/10
1014/1014 [==============================] - 5s 5ms/step - loss: 2.0425e-04 - acc: 1.0000 - val_loss: 1.6279e-04 - val_acc: 1.0000
Epoch 4/10
1014/1014 [==============================] - 5s 5ms/step - loss: 6.5555e-05 - acc: 1.0000 - val_loss: 1.6338e-04 - val_acc: 1.0000
Epoch 5/10
1014/1014 [==============================] - 5s 5ms/step - loss: 2.0829e-05 - acc: 1.0000 - val_loss: 5.2606e-05 - val_acc: 1.0000
Epoch 6/10
1014/1014 [==============================] - 5s 5ms/step - loss: 6.5786e-06 - acc: 1.0000 - val_loss: 2.3364e-05 - val_acc: 1.0000
Epoch 7/10
1014/1014 [==============================] - 5s 5ms/step - loss: 3.0762e-06 - acc: 1.0000 - val_loss: 

In [5]:
seed = 42
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(scaled_data, y):
  # create model
    print("making model")
    model_cnn = Sequential()
    model_cnn.add(Conv1D(64, 4, activation="relu", input_shape=(999,2)))
    model_cnn.add(MaxPooling1D(2))
    model_cnn.add(Conv1D(64, 4, activation="relu"))
    model_cnn.add(GlobalMaxPooling1D())
    model_cnn.add(Dense(1, activation="sigmoid"))
    # Compile model
    
    model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # Fit the model
    print("training model")
    history = model_cnn.fit(scaled_data[train], y[train], epochs=4)
    # evaluate the model
    print("evaluating model")
    scores = model_cnn.evaluate(scaled_data[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model_cnn.metrics_names[1], scores[1]*100))
    print("training accuracy: %.2f" %(history.history["acc"])[1])
    cvscores.append(scores[1] * 100)
    print("\n\n")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

making model
training model
Epoch 1/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.1775 - acc: 0.9661
Epoch 2/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.0210 - acc: 0.9971
Epoch 3/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.0062 - acc: 0.9988
Epoch 4/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.0022 - acc: 1.0000
evaluating model
acc: 100.00%
training accuracy: 1.00



making model
training model
Epoch 1/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.1788 - acc: 0.9776
Epoch 2/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.0195 - acc: 0.9983
Epoch 3/4
1739/1739 [==============================] - 12s 7ms/step - loss: 0.0058 - acc: 0.9983
Epoch 4/4
1739/1739 [==============================] - 11s 7ms/step - loss: 0.0021 - acc: 0.9994
evaluating model
acc: 100.00%
training accuracy: 1.00



making model
training model
Epoch 1/4
1739/1739 [=====

In [19]:
from keras import models
layer_outputs = [layer.output for layer in model_cnn.layers[:2]]
activation_model = models.Model(inputs=model_cnn.input, outputs=layer_outputs)

In [21]:
activations = activation_model.predict(X_train[0].reshape((1,999,2)))

In [23]:
import matplotlib.pyplot as plt
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

ImportError: No module named matplotlib.pyplot